# 用于研究vnpy的功能

In [5]:
# !pip install TA-Lib
# !pip install vnpy

In [1]:
import pandas as pd
import numpy as np
from abc import abstractmethod
import datetime

import vnpy

from const import *

In [2]:
start_str = "2021-12-01"
end_str = "2021-12-31"

symbol = "BTCUSDT"

## 数据准备

### 因子计算：每日最高涨幅

In [3]:
# 读取回测数据
daily_data_path = DATA_ROOT_PATH_KLINE + "/{}.csv".format(symbol)
df_daily = pd.read_csv(daily_data_path)

In [3]:
class DataReaderTemplate(object):
    """
    用于从数据库中读取数据，并进行初步的数据处理
    不同数据的数据处理请见具体函数说明

    functions：
    1. 格式处理
    2. 读取数据的对外接口
    """

    def drop_unnamed(self, data):
        if "Unnamed: 0" in data.columns:
            data = data.drop(columns=["Unnamed: 0"], axis=1)

        return data
    
    @abstractmethod
    def format_transfer(self, data):
        """
        用于修改数据格式
        """
        pass

    def read_data(self, file_name: str):
        data = pd.read_csv(self.root_path + '/{}.csv'.format(file_name))
        data = self.drop_unnamed(data)
        return self.format_transfer(data)

    pass

In [4]:
class KLiineDataReader(DataReaderTemplate):
    """
    用于读取分钟级k线数据
    """

    def __init__(self, root_path=DATA_ROOT_PATH_KLINE):
        """
        root_path: 该类数据的存储位置，与data_downloader中的路径保持一致
        """
        self.root_path = root_path
        pass

    def format_transfer(self, data):
        data = data.set_index(keys="open_time", drop=True)
        return data


class DailyPriceDataReader(DataReaderTemplate):
    """
    用于读取日频k线数据
    """

    def __init__(self, root_path=DATA_ROOT_PATH_DAILYPRICE) -> None:
        self.root_path = root_path
        pass

    def format_transfer(self, data):
        data.index = data["open_time"].apply(lambda x: x.split(" ")[0])
        return data

In [5]:
# 因子数据
df_daily = DailyPriceDataReader().read_data(file_name=symbol)

# 转换因子：单日动量：昨日涨，今日涨
series_ratio = (df_daily["close"].shift(1) - df_daily["close"].shift(2))/df_daily["close"].shift(2)
series_signal = np.where(series_ratio>0, 1, 0)
df_signal = pd.DataFrame(series_signal, columns=["factor"], index=df_daily.index)

df_signal.index = pd.to_datetime(df_signal.index)
df_signal["date"] = df_signal.index

# 回测数据
df_daily = DailyPriceDataReader().read_data(file_name=symbol)
df_daily.index = pd.to_datetime(df_daily.index)
df_daily["date"] = df_daily.index

In [6]:
df_daily

,open_time,open,high,low,close,volume,close_time,amount,number,call_volume,call_amount,ignore,date
open_time,,,,,,,,,,,,,
2021-12-01,2021-12-01 00:00:00,56950.56,59053.55,56458.01,57184.07,44956.636560,2021-12-01 23:59:59,2.584929e+09,1442301,21501.29463,1.236414e+09,0,2021-12-01
2021-12-02,2021-12-02 00:00:00,57184.07,57375.47,55777.77,56480.34,37574.059760,2021-12-02 23:59:59,2.127110e+09,1307890,18375.47518,1.040367e+09,0,2021-12-02
2021-12-03,2021-12-03 00:00:00,56484.26,57600.00,51680.00,53601.05,58927.690270,2021-12-03 23:59:59,3.246199e+09,1865063,27287.25751,1.505557e+09,0,2021-12-03
2021-12-04,2021-12-04 00:00:00,53601.05,53859.10,42000.30,49152.47,114203.373748,2021-12-04 23:59:59,5.495712e+09,3362771,52876.55238,2.544633e+09,0,2021-12-04
2021-12-05,2021-12-05 00:00:00,49152.46,49699.05,47727.21,49396.33,45580.820120,2021-12-05 23:59:59,2.231486e+09,1371839,22420.90324,1.097786e+09,0,2021-12-05
2021-12-06,2021-12-06 00:00:00,49396.32,50891.11,47100.00,50441.92,58571.215750,2021-12-06 23:59:59,2.851529e+09,1741956,28820.34753,1.403465e+09,0,2021-12-06
2021-12-07,2021-12-07 00:00:00,50441.91,51936.33,50039.74,50588.95,38253.468770,2021-12-07 23:59:59,1.950574e+09,1212350,18679.11099,9.525511e+08,0,2021-12-07
2021-12-08,2021-12-08 00:00:00,50588.95,51200.00,48600.00,50471.19,38425.924660,2021-12-08 23:59:59,1.925459e+09,1118225,18126.84362,9.084107e+08,0,2021-12-08
2021-12-09,2021-12-09 00:00:00,50471.19,50797.76,47320.00,47545.59,37692.686650,2021-12-09 23:59:59,1.841475e+09,1130599,18097.05960,8.840825e+08,0,2021-12-09


## 开始回测

In [2]:
# !pip install backtrader

In [7]:
import backtrader as bt

In [8]:
class MyCustomPandasData(bt.feeds.PandasData):
    lines = ('factor',)  # 添加新的数据项

    # 设置新的数据项的列索引
    params = (
        ('factor', -1),
    )

In [27]:
# 定义测试策略：传入信号为1，买，否则，卖
class TestStrategy(bt.Strategy):
 
    def log(self, txt, dt=None):
        ''' 记录策略信息'''
        dt = dt or self.datas[0].datetime.date(0)
 
    def __init__(self):
        # 应用第二个数据源的收盘价
        self.factor = self.datas[1].factor
        self.close = self.datas[0].close
        # self.factor = self.datas[0].close
 
    def next(self):
        # Simply log the closing price of the series from the reference
        # print(self.factor[0])
        if self.factor[0] == 1:
            self.buy(price=self.datas[0].close[-1], size=1)
        else:
            self.sell(price=self.datas[0].open[0], size=1)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # 订单已提交或被接受，不做处理
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}, , size: {order.executed.size}')
            elif order.issell():
                self.log(f'SELL EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}')

            self.bar_executed = len(self)  # 记录订单执行的时间点

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None  # 重置订单变量

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log(f'OPERATION PROFIT, GROSS: {trade.pnl}, NET: {trade.pnlcomm}')

    def log(self, txt, dt=None):
        '''Logging function for this strategy'''
        dt = dt or self.data.datetime.date(0)
        print(f'{dt.isoformat()} {txt}')


In [28]:
cerebro = bt.Cerebro()

# 增加一个策略
cerebro.addstrategy(TestStrategy)

# 定义策略回测的开始和结束时间
start_date = pd.to_datetime(start_str)
end_date = pd.to_datetime(end_str)

# 回测数据和因子数据准备
btdf_history = bt.feeds.PandasData(dataname=df_daily, fromdate=start_date, todate=end_date)
btdf_singal = MyCustomPandasData(dataname=df_signal, fromdate=start_date, todate=end_date)

cerebro.adddata(btdf_history)
cerebro.adddata(btdf_singal)

cerebro.broker.setcash(100000.0)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())



Starting Portfolio Value: 100000.00
2021-12-02 SELL EXECUTED, Price: 57184.07, Cost: -57184.07, Comm: 0.0
2021-12-03 SELL EXECUTED, Price: 56484.26, Cost: -56484.26, Comm: 0.0
2021-12-04 SELL EXECUTED, Price: 53601.05, Cost: -53601.05, Comm: 0.0
2021-12-05 SELL EXECUTED, Price: 49152.46, Cost: -49152.46, Comm: 0.0
2021-12-06 SELL EXECUTED, Price: 49396.32, Cost: -49396.32, Comm: 0.0
2021-12-07 BUY EXECUTED, Price: 50441.91, Cost: -53163.632, Comm: 0.0, , size: 1
2021-12-08 BUY EXECUTED, Price: 50588.95, Cost: -53163.632, Comm: 0.0, , size: 1
2021-12-09 BUY EXECUTED, Price: 50471.19, Cost: -53163.632, Comm: 0.0, , size: 1
2021-12-10 SELL EXECUTED, Price: 47535.9, Cost: -47535.9, Comm: 0.0
2021-12-11 SELL EXECUTED, Price: 47140.54, Cost: -47140.54, Comm: 0.0
2021-12-12 SELL EXECUTED, Price: 49389.99, Cost: -49389.99, Comm: 0.0
2021-12-13 BUY EXECUTED, Price: 50053.9, Cost: -50078.7388, Comm: 0.0, , size: 1
2021-12-14 BUY EXECUTED, Price: 46702.76, Cost: -50078.7388, Comm: 0.0, , size: 1
